# Testing Ximea Aquisition

### Imports and Params

In [1]:
import ximea_cam_aquire_save as xim
#import xca2 as xim
import imp
import run_analysis as ana
from ximea import xiapi
import numpy as np
import cv2
import matplotlib.pyplot as plt


In [2]:
collection_mins = 0.5
save_batchsize = 2500

### Capture for Ximea Cameras

https://www.ximea.com/support/wiki/apis/XiAPI_Manual

In [ ]:
imp.reload(xim)
save_folder_list = ['./test', './test2']
xim.ximea_acquire(save_folder_list, max_collection_mins=collection_mins, ims_per_file=save_batchsize)

SCENE_CAM Starting acquisition threads...
SCENE_CAM Opening Camera od
SCENE_CAM Aquiring Until Finished...
SCENE_CAM Recording Timestamp Syncronization Pre...
garbage collected! Queue size is [65]
garbage collected! Queue size is [183]


### How many Frames were Dropped?

In [ ]:
imp.reload(ana)
ana.count_missed_frames('./test/timestamps_od.tsv', 'od')
#ana.count_missed_frames('/tb_disk/test/timestamps_os.tsv', 'os')
# ana.count_missed_frames('/tb_disk/test/timestamps_cy.tsv', 'cy')

ana.plot_camera_timing('./test/timestamps_od.tsv', './test/timestamp_plot_od.png', 'od')
#ana.plot_camera_timing('/tb_disk/test/timestamps_os.tsv', '/tb_disk/test/timestamps_os.png', 'os')

ana.plot_camera_dframe('./test/timestamps_od.tsv', './test/dframe_plot_od.png', 'od')
#ana.plot_camera_dframe('/tb_disk/test/timestamps_os.tsv', '/tb_disk/test/timestamps_os.png', 'od')


### Do Images Look Good?

In [ ]:
imgfile = 'test/od/frames_100_199.bin'
imc_od = ana.bin_to_im(imgfile, 10, (1544,2064))
imc_od = (imc_od/np.max(imc_od))
cv2.imwrite('./testim.png', imc_od)
#imc_os = bin_to_im('test/os/frame_8000.bin')
plt.imshow(imc_od)
plt.title('OD')
plt.show()

cv2.imwrite('./testim_norm.png', cv2.cvtColor(((imc_od*255).astype('uint8')), cv2.COLOR_RGB2BGR))
plt.hist(imc_od.flatten(), bins=100);
#plt.imshow(imc_os)
#plt.title('OS')
#plt.show()

In [ ]:
import matplotlib.pyplot as plt
imp.reload(ana)
imc_od2 = ana.bin_to_im(imgfile, 1, (1544,2064))
imc_od2 = (imc_od2/np.max(imc_od2))
cv2.imwrite('./testim.png', imc_od2)
#imc_os = bin_to_im('test/os/frame_8000.bin')
plt.imshow(imc_od2)
plt.title('OD')
plt.show()

cv2.imwrite('./testim_norm2.png', cv2.cvtColor(((imc_od2*255).astype('uint8')), cv2.COLOR_RGB2BGR))
plt.hist(imc_od2.flatten(), bins=100);
#plt.imshow(imc_os)
#plt.title('OS')
#plt.show()

In [ ]:
diff = imc_od2 - imc_od
diff = np.abs(diff/np.max(diff))
plt.imshow(diff)
plt.show()
plt.hist(diff.flatten(),bins=100)
#cv2.imwrite('./testim_norm_diff.png', cv2.cvtColor(((diff*255).astype('uint8')), cv2.COLOR_RGB2BGR))
cv2.imwrite('./testim_norm_diff.png', cv2.cvtColor(((diff*255).astype('uint8')), cv2.COLOR_RGB2BGR))


### Test checking parameters of camera

In [ ]:
cam_id = "XECAS1922000"
import ctypes
from collections import namedtuple
import multiprocessing as mp

pout, pin = mp.Pipe(duplex=False)

camera = xiapi.Camera()
camera.open_device_by_SN(cam_id)
xim.apply_cam_settings(camera, 'od.yaml')

#params = ['limit_bandwidth_mode', 'limit_bandwidth', 'framerate', 'transport_data_target']
#for p in params:
#    print(p, camera.get_param(p))

camera.start_acquisition()    
image = xiapi.Image()
camera.get_image(image)
#data = (ctypes.c_uint*image.bp_size).from_address(image.bp)
data = ctypes.cast(image.bp, ctypes.POINTER(ctypes.c_uint))
frame_data = namedtuple("frame_data", "raw_data nframe tsSec tsUSec")

d = frame_data(data.contents,image.nframe,
                    image.tsSec,
                   image.tsUSec)
pin.send(d)
#print(image.transport_frm)

camera.close_device()

In [ ]:
a = pout.recv()

In [ ]:
a

In [ ]:
camera.close_device()


In [ ]:
ana.plot_camera_timing('./test/timestamps_od.tsv', './test/timestamp_plot_od.png', 'od')
ana.plot_camera_timing('./test/timestamps_os.tsv', './test/timestamp_plot_os.png', 'os')
ana.plot_camera_timing('/tb_disk/test/timestamps_cy.tsv', './test/timestamp_plot_cy.png', 'cy')

ana.plot_camera_dframe('./test/timestamps_od.tsv', './test/dframe_plot_od.png', 'od')
ana.plot_camera_dframe('./test/timestamps_os.tsv', './test/dframe_plot_os.png', 'os')
ana.plot_camera_dframe('/tb_disk/test/timestamps_cy.tsv', './test/dframe_plot_cy.png', 'cy')


## Test HD write speed

In [ ]:
import os
import time
import cProfile
import mmap

def ioTest():
    m = mmap.mmap(-1, 10*1024*1024)
    block = bytes('A'*10*1024*1024, 'utf-8')
    m.write(block) 
    filename = './test.bin'
    f = os.open(filename, os.O_WRONLY | os.O_CREAT | os.O_TRUNC | os.O_SYNC | os.O_DIRECT)
    start = time.time()
    for x in range(500):
        os.write(f,m)
    os.close(f)
    transferTime_sec = time.time() - start
    msg = 'Wrote {:0f}MB in {:0.03f}s.'
    print(msg.format(os.stat(filename).st_size/1024/1024,
                     transferTime_sec))
cProfile.run('ioTest()')

In [ ]:
import numpy as np
dims = (12, 20)
np.prod(dims)

In [ ]:
import ximea_cam_aquire_save as aquire
import imp
import numpy as np
import matplotlib.pyplot as plt
import imageio
import cv2

## Bulk Collection

In [ ]:
#settings:
num_frames = 10
cam_exposure = 1000
cam_gain = 20

In [ ]:
imp.reload(aquire)
aquire.aquire_save_fullset(num_frames=num_frames, verbose=True, cam_exposure=cam_exposure, cam_gain=cam_gain)


In [ ]:
ims = []
for i in range(num_frames):
    imb = []
    with open(f'capture/batch/data_raw_bytes_{i}.bin', 'rb') as f:
        byte = f.read(1)
        while(byte):
            byte = f.read(1)
            imb.append(int.from_bytes(byte,'big'))
        ims.append(np.array(imb).reshape((1544,2064)))
        #ims.append(np.array(imb).reshape((1544,2064)))
        #ims.append(np.array(map('|S1',list(imb))).reshape((1544,2064)))
        f.close()
for i in range(len(ims)):
    print(np.shape(np.uint8(ims[i])))
    #ims[i] = cv2.cvtColor(np.uint8(ims[i]), cv2.COLOR_BayerGR2RGB)
    ims[i] = np.uint8(ims[i])
    print(np.shape(ims[i]))
    imageio.imwrite(f'capture/batch/img_{i}.jpg', ims[i])

In [ ]:
im = ims[0]
#im = cv2.cvtColor(np.uint8(im), cv2.COLOR_BayerGR2RGB)
plt.figure(figsize=(10,12))
plt.imshow(im)

In [ ]:
from datetime import datetime
from PIL import Image
#import time
 
def imgDate(fn):
    "returns the image date from image (if available)\nfrom Orthallelous"
    std_fmt = '%Y:%m:%d %H:%M:%S.%f'
    # for subsecond prec, see doi.org/10.3189/2013JoG12J126 , sect. 2.2, 2.3
    tags = [(36867, 37521),  # (DateTimeOriginal, SubsecTimeOriginal)
            (36868, 37522),  # (DateTimeDigitized, SubsecTimeOriginal)
            (306, 37520), ]  # (DateTime, SubsecTime)
    exif = Image.open(fn)._getexif()
 
    for t in tags:
        dat_stmp = exif.get(t[0])
        sub_stmp = exif.get(t[1], 0)
 
        # PIL.PILLOW_VERSION >= 3.0 returns a tuple
        dat_stmp = dat_stmp[0] if type(dat_stmp) == tuple else dat_stmp
        sub_stmp = sub_stmp[0] if type(sub_stmp) == tuple else sub_stmp
        if dat_stmp != None: break
 
    if dat_stmp == None: return None
    full = '{}.{}'.format(dat_stmp, sub_stmp)
    T = datetime.strptime(full, std_fmt)
    T = time.mktime(time.strptime(dT, '%Y:%m:%d %H:%M:%S')) + float('0.%s'%sub)
    return T

In [ ]:
Image.open('capture/batch/img_0.jpg')[36867]

In [ ]:
#line profile experiment running
imp.reload(runexp)
lp = LineProfiler()
# lp_wrapper = lp(runexp.run_experiment)
# lp_wrapper(subject_name=subject, 
#                       task_name=task, 
#                       exp_type=exp,
#                       save_dir=capture_dir,
#                       max_frames=frames)
lp_wrapper = lp(runexp.run_ximea_aquisition)
lp_wrapper('./capture/line_profile_test', 200, 1000)
lp.print_stats()

In [ ]:
imp.reload(runana)
runana.run_analysis(subject_name=subject, 
                      task_name=task, 
                      exp_type=exp,
                      read_dir=capture_dir,
                      save_dir=analysis_dir,
                      run_timestamp_stats=True,
                      run_convert_ims=True
                      )

In [ ]:
im = cv2.imread('./analysis/test_subject/test_task/exp/scene_camera/cam_od/frame_0.png')
plt.imshow(im)
plt.show()
plt.hist(im.flatten())

In [ ]:

f= 'capture/test_subject/test_task/exp/scene_camera/cam_od/frame_0.bin'
a = []
# # for uint16
# with open(f, 'rb') as fn:
#     bss = fn.read(2)
#     while(bss):
#         bss = fn.read(1)
#         bbs = fn.read(1)
#         bs = int.from_bytes(bss,'big')
#         bb = int.from_bytes(bbs,'big')
#         s = 256*bb+bs
#         a.append(s)

# for uint8
with open(f, 'rb') as fn:
    bs = fn.read(1)
    while(bs):
        bs = fn.read(1)
        bs = int.from_bytes(bs,'big')
        a.append(bs)

a = np.array(a)

In [ ]:
plt.hist(np.array(a).flatten(), bins=1000);

In [ ]:
im = a.reshape((1544,2064))
imc = cv2.cvtColor(np.uint16(im), cv2.COLOR_BayerGR2RGB)

In [ ]:
plt.imshow(im)
plt.show()
plt.imshow(imc)
plt.show()

In [ ]:
cv2.imwrite('test.png',imc.astype('float64'))
imcr = cv2.imread('test.png')
plt.imshow(imcr)
plt.show()
imcrw = cv2.balanceWhite(imcr)
plt.imshow(imcrw)
plt.show()


In [ ]:
plt.hist(np.array(imc).flatten(), bins=1000);

In [ ]:
plt.hist(np.array(imc.astype('float64')).flatten(), bins=1000);

In [ ]:
plt.hist(np.array(imcr).flatten(), bins=1000);